In [1]:
using Pkg
Pkg.activate(".")
caminho_do_pacote = "/home/luizfreire/Documents/mestrado/powerModelsExemplos/PowerModelsDistributionDev"
Pkg.develop(PackageSpec(path=caminho_do_pacote))

  Activating project at `~/Documents/mestrado/powerModelsExemplos`
   Resolving package versions...
  No Changes to `~/Documents/mestrado/powerModelsExemplos/Project.toml`
  No Changes to `~/Documents/mestrado/powerModelsExemplos/Manifest.toml`


In [2]:
using PowerModelsDistribution
using Ipopt
using JuMP
using PowerPlots
using Plots

include("utils/load_data.jl")
results_path = "results/2025-04-21_solar_carga_armazenamento_opf_cost/"

solver = optimizer_with_attributes(
    Ipopt.Optimizer,
    "max_iter" => 50000,
    "tol" => 1e-8)


Precompiling PowerModelsDistribution
  ✓ PowerModelsDistribution
  1 dependency successfully precompiled in 12 seconds. 71 already precompiled.


MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("max_iter") => 50000, MathOptInterface.RawOptimizerAttribute("tol") => 1.0e-8])

In [3]:
eng_model = PowerModelsDistribution.Model()

add_voltage_source!(
    eng_model,
    "voltage_source_1",
    "bus_1",
    [1, 2, 3, 4],
    configuration=WYE,
    vm=[2.4, 2.4, 2.4, 0.0],
    va=[0.0, -120.0, 120.0, 0.0],
    rs=zeros(4),
    xs=zeros(4),
    status=ENABLED
)

add_line!(eng_model,
          "line_1",
          "bus_1",
          "bus_2",
          [1, 2, 3],
          [1, 2, 3],
          rs=zeros(3, 3),
          xs=zeros(3, 3))

add_prosumer!(eng_model,
              "prosumer_1",
              "bus_2",
              [1, 2, 3, 4];
              configuration=WYE,
              pg=[2500, 2500, 2500, 0],
              pd=[1800, 1800, 1800, 0],
              ps=[1000, 1000, 1000, 0],
              energy=20000,
              energy_ub=100000,
              charge_ub=7000,
              discharge_ub=7000,
              charge_efficiency=1.0,
              discharge_efficiency=1.0,
              cost=1)

# reference = PowerModelsDistribution.parse_file("4Bus-DY-Bal/4Bus-DY-Bal.DSS")
#delete!(eng_model["settings"]["base_bus"])
eng_model["settings"]["vbases_default"] = Dict("bus_1" => 2.4)

Dict{String, Float64} with 1 entry:
  "bus_1" => 2.4

In [4]:
eng_model["prosumer"]["prosumer_1"]

Dict{String, Any} with 16 entries:
  "pg"                   => [2500, 2500, 2500, 0]
  "connections"          => [1, 2, 3, 4]
  "energy"               => 20000
  "bus"                  => "bus_2"
  "charge_ub"            => 7000
  "configuration"        => WYE
  "energy_ub"            => 100000
  "status"               => ENABLED
  "source_id"            => "prosumer.prosumer_1"
  "charge_efficiency"    => 1.0
  "discharge_ub"         => 7000
  "ps"                   => [1000, 1000, 1000, 0]
  "cost"                 => 1
  "control_mode"         => FREQUENCYDROOP
  "discharge_efficiency" => 1.0
  "pd"                   => [1800, 1800, 1800, 0]

In [5]:
transform_data_model(eng_model)["prosumer"]["1"]

[ PowerModelsDistribution | Info ] : no active generators found at bus 2, updating to bus type from 2 to 1


Dict{String, Any} with 30 entries:
  "energy_rating"     => 100000
  "pg"                => [2500, 2500, 2500, 0]
  "connections"       => [1, 2, 3, 4]
  "b"                 => 0.0
  "energy"            => 20000
  "a"                 => 0.0
  "configuration"     => WYE
  "qs"                => [0.0, 0.0, 0.0]
  "name"              => "prosumer_1"
  "prosumer_bus"      => 2
  "qg"                => [0.0, 0.0, 0.0]
  "thermal_rating"    => Inf
  "pmax"              => [Inf, Inf, Inf]
  "qd"                => [0.0, 0.0, 0.0]
  "source_id"         => "prosumer.prosumer_1"
  "discharge_rating"  => 7000
  "alpha"             => 1.0
  "charge_efficiency" => 0.01
  "index"             => 1
  "ps"                => [1000, 1000, 1000, 0]
  "beta"              => 1.0
  "cost"              => 1
  "qmax"              => [Inf, Inf, Inf]
  "prosumer_status"   => 1
  "qmin"              => [-Inf, -Inf, -Inf]
  ⋮                   => ⋮

In [6]:
result = solve_mc_model(eng_model, ACPUPowerModel, solver, build_mc_tes_opf)

[ PowerModelsDistribution | Info ] : no active generators found at bus 2, updating to bus type from 2 to 1



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       21
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       31
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        4
                     variables with only upper bounds:        0
Total number of equality constraints.................:       15
Total number of inequality c

Dict{String, Any} with 8 entries:
  "solve_time"         => 0.00664401
  "optimizer"          => "Ipopt"
  "termination_status" => LOCALLY_SOLVED
  "dual_status"        => FEASIBLE_POINT
  "primal_status"      => FEASIBLE_POINT
  "objective"          => -28000.0
  "solution"           => Dict{String, Any}("voltage_source"=>Dict{String, Any}…
  "objective_lb"       => -Inf

In [7]:
result["solution"]["prosumer"]

Dict{String, Any} with 1 entry:
  "prosumer_1" => Dict{String, Any}("ps"=>[-7000.0, -7000.0, -7000.0, -7000.0])